# Dataset Spark coding

## Connecting to a server

In [6]:
# Closing all active Kerberos connections (tickets):
# (effectively logging out from Kerberos-secured services)
#!kdestroy

# Starting a Kerberos connection:
#path = "Login/"
#!kinit -kt f"{path}/Jakub.Cajzl.keytab" Jakub.Cajzl

# Displaying current login:
!klist


Current LogonId is 0:0x133b8d

Cached Tickets: (1)

#0>	Client: Jakub.Cajzl @ ADASTRACORP.NET
	Server: krbtgt/ADASTRACORP.NET @ ADASTRACORP.NET
	KerbTicket Encryption Type: AES-256-CTS-HMAC-SHA1-96
	Ticket Flags 0x40e10000 -> forwardable renewable initial pre_authent name_canonicalize 
	Start Time: 6/11/2024 12:01:23 (local)
	End Time:   6/11/2024 22:01:23 (local)
	Renew Time: 6/18/2024 12:01:23 (local)
	Session Key Type: AES-256-CTS-HMAC-SHA1-96
	Cache Flags: 0x1 -> PRIMARY 
	Kdc Called: czprgprdw05-new.adastracorp.net


## Libraries

In [7]:
# Importing libraries:
import sys
import pyspark
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession, DataFrame, functions, types, SQLContext

from pyspark.sql.functions import *
from pyspark.sql.types import *

from datetime import datetime, date
import pandas as pd

# It is better to load pySpark version of Pandas as 'ps' instead of 'pd'
# because pySpark has differences in their version of Pandas 
# (mainly because pySpark is masivelly parallelized and has many 
# parallelization parameters).
from pyspark import pandas as ps

import findspark
findspark.init()

c:\Users\Jakub.Cajzl\AppData\Local\Programs\Python\Python312\Lib\site-packages\pyspark\pandas\__init__.py:50: UserWarning: 'PYARROW_IGNORE_TIMEZONE' environment variable was not set. It is required to set this environment variable to '1' in both driver and executor sides if you use pyarrow>=2.0.0. pandas-on-Spark will set it for you but it does not work if there is a Spark context already launched.
  warnings.warn(


In [10]:
# Starting a Spark session:
spark = SparkSession.builder.appName("code01").getOrCreate()

# Spark Context:
# sc = spark.SparkContext
# sqlContext = SQLContext(sc)

## Basics

### DataFrame creation

#### 1. From list of rows

In [12]:
data = [
    (1, "Alice", 29),
    (2, "Bob", 31),
    (3, "Cathy", 25),
    (4, "David", 35),
    (5, "Eva", 28)
]

columns = ["id", "name", "age"]

df = spark.createDataFrame(data,columns)

In [14]:
df.printSchema()

root
 |-- id: long (nullable = true)
 |-- name: string (nullable = true)
 |-- age: long (nullable = true)



In [16]:
print(df)

DataFrame[id: bigint, name: string, age: bigint]


In [19]:
df.count()

ConnectionRefusedError: [WinError 10061] No connection could be made because the target machine actively refused it

#### 2. With an explicit schema

In [7]:
df = spark.createDataFrame([
    (1, 2., 'string1', date(2000, 1, 1), datetime(2000, 1, 1, 12, 0)),
    (2, 3., 'string2', date(2000, 2, 1), datetime(2000, 1, 2, 12, 0)),
    (3, 4., 'string3', date(2000, 3, 1), datetime(2000, 1, 3, 12, 0))
], schema='a long, b double, c string, d date, e timestamp')

df.show()

Py4JJavaError: An error occurred while calling o80.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 0.0 failed 1 times, most recent failure: Lost task 0.0 in stage 0.0 (TID 0) (Cloud-CQKD9G3.adastracorp.net executor driver): org.apache.spark.SparkException: Python worker failed to connect back.
	at org.apache.spark.api.python.PythonWorkerFactory.createSimpleWorker(PythonWorkerFactory.scala:203)
	at org.apache.spark.api.python.PythonWorkerFactory.create(PythonWorkerFactory.scala:109)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:124)
	at org.apache.spark.api.python.BasePythonRunner.compute(PythonRunner.scala:174)
	at org.apache.spark.api.python.PythonRDD.compute(PythonRDD.scala:67)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoint(RDD.scala:367)
	at org.apache.spark.rdd.RDD.iterator(RDD.scala:331)
	at org.apache.spark.rdd.MapPartitionsRDD.compute(MapPartitionsRDD.scala:52)
	at org.apache.spark.rdd.RDD.computeO

#### 3. From a pandas DataFrame

In [10]:
pandas_df = pd.DataFrame({
    'a': [1, 2, 3],
    'b': [2., 3., 4.],
    'c': ['string1', 'string2', 'string3'],
    'd': [date(2000, 1, 1), date(2000, 2, 1), date(2000, 3, 1)],
    'e': [datetime(2000, 1, 1, 12, 0), datetime(2000, 1, 2, 12, 0), datetime(2000, 1, 3, 12, 0)]
})
df = spark.createDataFrame(pandas_df)
df

DataFrame[a: bigint, b: double, c: string, d: date, e: timestamp]

#### Showing DataFrame and schema

In [12]:
# All DataFrames above result the same:
df.printSchema()

root
 |-- a: long (nullable = true)
 |-- b: double (nullable = true)
 |-- c: string (nullable = true)
 |-- d: date (nullable = true)
 |-- e: timestamp (nullable = true)



In [13]:
df.show()

Py4JJavaError: An error occurred while calling o133.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 2.0 failed 1 times, most recent failure: Lost task 0.0 in stage 2.0 (TID 2) (Cloud-CQKD9G3.adastracorp.net executor driver): org.apache.spark.SparkException: Python worker failed to connect back.
	at org.apache.spark.api.python.PythonWorkerFactory.createSimpleWorker(PythonWorkerFactory.scala:203)
	at org.apache.spark.api.python.PythonWorkerFactory.create(PythonWorkerFactory.scala:109)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:124)
	at org.apache.spark.api.python.BasePythonRunner.compute(PythonRunner.scala:174)
	at org.apache.spark.api.python.PythonRDD.compute(PythonRDD.scala:67)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoint(RDD.scala:367)
	at org.apache.spark.rdd.RDD.iterator(RDD.scala:331)
	at org.apache.spark.rdd.MapPartitionsRDD.compute(MapPartitionsRDD.scala:52)
	at org.apache.spark.rdd.RDD.compute

#### Viewing data as Python DataFrame (eager evaluation)

In [15]:
spark.conf.set('spark.sql.repl.eagerEval.enabled', True)
df

Py4JJavaError: An error occurred while calling o133.htmlString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 3.0 failed 1 times, most recent failure: Lost task 0.0 in stage 3.0 (TID 3) (Cloud-CQKD9G3.adastracorp.net executor driver): org.apache.spark.SparkException: Python worker failed to connect back.
	at org.apache.spark.api.python.PythonWorkerFactory.createSimpleWorker(PythonWorkerFactory.scala:203)
	at org.apache.spark.api.python.PythonWorkerFactory.create(PythonWorkerFactory.scala:109)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:124)
	at org.apache.spark.api.python.BasePythonRunner.compute(PythonRunner.scala:174)
	at org.apache.spark.api.python.PythonRDD.compute(PythonRDD.scala:67)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoint(RDD.scala:367)
	at org.apache.spark.rdd.RDD.iterator(RDD.scala:331)
	at org.apache.spark.rdd.MapPartitionsRDD.compute(MapPartitionsRDD.scala:52)
	at org.apache.spark.rdd.RDD.compute

DataFrame[a: bigint, b: double, c: string, d: date, e: timestamp]

## Transformations

**Transformations:**
- map(func)
- filter(func)
- flatMap(func)
- mapPartitons(func)
- sample(withReplacement, fraction, seed)
- union(otherRDD)
- intersection(otherRDD)
- distinct()
- groupByKey()
- reduceByKey(func)
- sortByKey()
- join(otherRDD)
- ...

## Actions

**Actions:**
- reduce(func)
- count()
- first()
- take(n)
- takeSample(withReplacement, n)
- saveAsTextFile(path)
- countByKey()
- foreach(func)

In [ ]:
# Grouping, counting and sorting:
display(spark_df.groupby("column").count().orderBy("count", ascending=False))

# Or alternatively (Pandas syntax):
spark_df["column"].value_counts()

## Pandas on PySpark

In [ ]:
# Example of CSV file read:
df = ps.read_csv(f'{directory}/dataset.csv', inferSchema=True, multiLine=True, escape='"')

In [ ]:
# Converting a Pandas DataFrame to Spark DataFrame:

# First way:
df = ps.DataFrame(spark_DataFrame)
display(df)

# Second way:
df = spark_DataFrame.to_pandas_on_spark()
display(df)

Note: If a pandas-on-Spark DataFrame is converted to a Spark DataFrame and then back to pandas-on-Spark, it will **lose the index** information and the **original index** will be **converted into** a normal **column**.

### SQL on Pandas

In [ ]:
ps.sql("SELECT distinct(column) FROM {spark_DataFrame}")